In [4]:
%matplotlib inline


What is PyTorch?
================

It’s a Python based scientific computing package targeted at two sets of
audiences:

-  A replacement for NumPy to use the power of GPUs
-  a deep learning research platform that provides maximum flexibility
   and speed

Getting Started
---------------

Tensors
^^^^^^^

Tensors are similar to NumPy’s ndarrays, with the addition being that
Tensors can also be used on a GPU to accelerate computing.



In [1]:
from __future__ import print_function
import torch

Construct a 5x3 matrix, uninitialized:



In [2]:
x = torch.Tensor(5, 3)
print(x)


 0  0  0
 0  0  0
 0  0  0
 0  0  0
 0  0  0
[torch.FloatTensor of size 5x3]



Construct a randomly initialized matrix:



In [3]:
x = torch.rand(5, 3)
print(x)


 0.2430  0.8310  0.8615
 0.5551  0.4359  0.6892
 0.3522  0.9560  0.6339
 0.4599  0.8639  0.4027
 0.1889  0.1999  0.3252
[torch.FloatTensor of size 5x3]



Get its size:



In [4]:
print(x.size())

torch.Size([5, 3])


<div class="alert alert-info"><h4>Note</h4><p>``torch.Size`` is in fact a tuple, so it supports all tuple operations.</p></div>

Operations
^^^^^^^^^^
There are multiple syntaxes for operations. In the following
example, we will take a look at the addition operation.

Addition: syntax 1



In [5]:
y = torch.rand(5, 3)
print(x + y)


 0.7212  0.9637  1.4551
 0.8023  0.5352  0.7564
 0.5210  1.6889  1.3869
 1.3409  0.8769  0.4271
 0.8651  0.2870  0.3400
[torch.FloatTensor of size 5x3]



Addition: syntax 2



In [6]:
print(torch.add(x, y))


 0.7212  0.9637  1.4551
 0.8023  0.5352  0.7564
 0.5210  1.6889  1.3869
 1.3409  0.8769  0.4271
 0.8651  0.2870  0.3400
[torch.FloatTensor of size 5x3]



Addition: providing an output tensor as argument



In [7]:
result = torch.Tensor(5, 3)
torch.add(x, y, out=result)
print(result)


 0.7212  0.9637  1.4551
 0.8023  0.5352  0.7564
 0.5210  1.6889  1.3869
 1.3409  0.8769  0.4271
 0.8651  0.2870  0.3400
[torch.FloatTensor of size 5x3]



Addition: in-place



In [8]:
# adds x to y
y.add_(x)
print(y)


 0.7212  0.9637  1.4551
 0.8023  0.5352  0.7564
 0.5210  1.6889  1.3869
 1.3409  0.8769  0.4271
 0.8651  0.2870  0.3400
[torch.FloatTensor of size 5x3]



<div class="alert alert-info"><h4>Note</h4><p>Any operation that mutates a tensor in-place is post-fixed with an ``_``.
    For example: ``x.copy_(y)``, ``x.t_()``, will change ``x``.</p></div>

You can use standard NumPy-like indexing with all bells and whistles!



In [9]:
print(x[:, 1])


 0.8310
 0.4359
 0.9560
 0.8639
 0.1999
[torch.FloatTensor of size 5]



Resizing: If you want to resize/reshape tensor, you can use ``torch.view``:



In [10]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


**Read later:**


  100+ Tensor operations, including transposing, indexing, slicing,
  mathematical operations, linear algebra, random numbers, etc.,
  are described
  `here <http://pytorch.org/docs/torch>`_.

NumPy Bridge
------------

Converting a Torch Tensor to a NumPy array and vice versa is a breeze.

The Torch Tensor and NumPy array will share their underlying memory
locations, and changing one will change the other.

Converting a Torch Tensor to a NumPy Array
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^



In [11]:
a = torch.ones(5)
print(a)


 1
 1
 1
 1
 1
[torch.FloatTensor of size 5]



In [12]:
b = a.numpy()
print(b)

[ 1.  1.  1.  1.  1.]


See how the numpy array changed in value.



In [13]:
a.add_(1)
print(a)
print(b)


 2
 2
 2
 2
 2
[torch.FloatTensor of size 5]

[ 2.  2.  2.  2.  2.]


Converting NumPy Array to Torch Tensor
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
See how changing the np array changed the Torch Tensor automatically



In [14]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[ 2.  2.  2.  2.  2.]

 2
 2
 2
 2
 2
[torch.DoubleTensor of size 5]



All the Tensors on the CPU except a CharTensor support converting to
NumPy and back.

CUDA Tensors
------------

Tensors can be moved onto GPU using the ``.cuda`` method.



In [15]:
# let us run this cell only if CUDA is available
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y

C:\Users\ab9bh\Anaconda3\envs\pytorch\lib\site-packages\torch\tensor.py:293: UserWarning: self and other not broadcastable, but have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add(other)


In [16]:
torch.cuda.is_available()

True

### CPU

In [17]:
import numpy as np
from timeit import default_timer as timer

def pow(a, b, c):
    for i in range(a.size):
         c[i] = a[i] ** b[i]

def main():
    vec_size = 100000000

    a = b = np.array(np.random.sample(vec_size), dtype=np.float32)
    c = np.zeros(vec_size, dtype=np.float32)

    start = timer()
    pow(a, b, c)
    duration = timer() - start

    print(duration)

if __name__ == '__main__':
    main()

44.82697204822282


### GPU

In [19]:
import numpy as np
from timeit import default_timer as timer
from numba import vectorize

@vectorize(['float32(float32, float32)'], target='cuda')
def pow(a, b):
    return a ** b

def main():
    vec_size = 100000000

    a = b = np.array(np.random.sample(vec_size), dtype=np.float32)
    c = np.zeros(vec_size, dtype=np.float32)

    start = timer()
    c = pow(a, b)
    duration = timer() - start

    print(duration)

if __name__ == '__main__':
    main()

NvvmSupportError: libNVVM cannot be found. Do `conda install cudatoolkit`:
library nvvm not found

In [20]:
torch.cuda.is_available()

True